In [50]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

Select 20 categoies

In [51]:
import os
import shutil

# Source and destination paths
source_path = '/Users/angelaliang/Duke University /Duke Classes/F24/ME 555 CEE 590/Project/datasets/data/images/'
# Get a list of all subdirectories (categories)
categories = sorted([d for d in os.listdir(source_path) if os.path.isdir(os.path.join(source_path, d))])

print(f"Total Categories: {len(categories)}")
print("Categories:")
print(categories)

Total Categories: 101
Categories:
['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_

In [52]:
print(len(categories))

101


In [53]:
data_path = '/Users/angelaliang/Duke University /Duke Classes/F24/ME 555 CEE 590/Project/datasets/data/new'

# select 20 categories
selected_categories = categories[0:20]

# Copy selected categories to the new dataset folder
for category in selected_categories:
    src = os.path.join(source_path, category)
    dst = os.path.join(data_path, category)
    if os.path.exists(src):
        shutil.copytree(src, dst)
print("Dataset reduced to 20 categories!")

FileExistsError: [Errno 17] File exists: '/Users/angelaliang/Duke University /Duke Classes/F24/ME 555 CEE 590/Project/datasets/data/new/apple_pie'

In [ ]:
print(selected_categories)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla']


Model

In [ ]:
# Dataset directory
dataset_path = data_path

# Image dimensions (299x299 for InceptionV3 or 224x224 for ResNet50)
img_size = (299, 299)
batch_size = 32


# Enhances the training dataset by applying random transformations to the images, making the model more robust to variations. 
# Data Augmentation and Data Generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split 20% for validation
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)



# Load the InceptionV3 model with ImageNet weights, excluding the top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(*img_size, 3))

# Freeze all layers in the base model to retain pre-trained features
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(20, activation='softmax')(x)  # 20 categories for the output layer

# Final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model with categorical cross-entropy loss
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()


Found 16000 images belonging to 20 classes.
Found 4000 images belonging to 20 classes.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activa

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:

# Set up early stopping and model checkpoint
# callbacks = [
#     EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
#     ModelCheckpoint('best_food20_model.keras', save_best_only=True) 
# ]
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_food20_model.h5', save_best_only=True)  # Explicitly use .h5 extension
]


# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Adjust based on validation performance
    validation_data=validation_generator,
    callbacks=callbacks
)


Epoch 1/10
500/500 [==============================] - ETA: 0s - loss: 1.2318 - accuracy: 0.6225

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


500/500 [==============================] - 1784s 4s/step - loss: 1.2318 - accuracy: 0.6225 - val_loss: 1.1289 - val_accuracy: 0.6612
Epoch 2/10
500/500 [==============================] - 1607s 3s/step - loss: 1.1926 - accuracy: 0.6396 - val_loss: 1.1091 - val_accuracy: 0.6708
Epoch 3/10
500/500 [==============================] - 1574s 3s/step - loss: 1.1561 - accuracy: 0.6453 - val_loss: 1.0968 - val_accuracy: 0.6650
Epoch 4/10
500/500 [==============================] - 1503s 3s/step - loss: 1.1278 - accuracy: 0.6562 - val_loss: 1.0757 - val_accuracy: 0.6808
Epoch 5/10
500/500 [==============================] - 1601s 3s/step - loss: 1.0982 - accuracy: 0.6612 - val_loss: 1.0723 - val_accuracy: 0.6735
Epoch 6/10
500/500 [==============================] - 5019s 10s/step - loss: 1.0764 - accuracy: 0.6719 - val_loss: 1.0444 - val_accuracy: 0.6842
Epoch 7/10
500/500 [==============================] - 9611s 19s/step - loss: 1.0571 - accuracy: 0.6747 - val_loss: 1.0440 - val_accuracy: 0.6752
E

Evaluation

In [ ]:
# Unfreeze the last 30 layers of the base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Re-compile with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_finetune = model.fit(
    train_generator,
    epochs=10,  # Adjust as needed
    validation_data=validation_generator,
    callbacks=callbacks
)



Epoch 1/10
500/500 [==============================] - 1503s 3s/step - loss: 1.0643 - accuracy: 0.6769 - val_loss: 0.9926 - val_accuracy: 0.7053
Epoch 2/10
500/500 [==============================] - 1490s 3s/step - loss: 0.9687 - accuracy: 0.7053 - val_loss: 0.9428 - val_accuracy: 0.7205
Epoch 3/10
500/500 [==============================] - 1606s 3s/step - loss: 0.9189 - accuracy: 0.7169 - val_loss: 0.9185 - val_accuracy: 0.7212
Epoch 4/10
500/500 [==============================] - 1727s 3s/step - loss: 0.8876 - accuracy: 0.7269 - val_loss: 0.8895 - val_accuracy: 0.7347
Epoch 5/10
500/500 [==============================] - 1605s 3s/step - loss: 0.8520 - accuracy: 0.7330 - val_loss: 0.9023 - val_accuracy: 0.7222
Epoch 6/10
500/500 [==============================] - 1624s 3s/step - loss: 0.8268 - accuracy: 0.7454 - val_loss: 0.8740 - val_accuracy: 0.7415
Epoch 7/10
500/500 [==============================] - 1778s 4s/step - loss: 0.8089 - accuracy: 0.7522 - val_loss: 0.8721 - val_accuracy:

In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


125/125 [==============================] - 280s 2s/step - loss: 0.8352 - accuracy: 0.7458
Validation Accuracy: 74.58%


In [ ]:
model.save('fine_tuned_food20_model.h5')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Use the saved model 

from tensorflow.keras.models import load_model

# Load the entire model
model = load_model('fine_tuned_food20_model.h5')


# Portion Size Estimation 
Placeholder estimation for portion size

In [ ]:
import numpy as np
import pandas as pd

In [58]:
# selected_categories

# Base portion size (in grams) and variation factor
base_portion = 200
variation_factor = 0.5  # ±50%

# Randomly generate portion sizes for all categories
def generate_portion_ranges(categories, base, variation):
    portion_ranges = {}
    for category in categories:
        min_portion = np.random.uniform(base * (1 - variation), base * (1 - variation / 2))
        max_portion = np.random.uniform(base * (1 + variation / 2), base * (1 + variation))
        portion_ranges[category] = (round(min_portion), round(max_portion))
    return portion_ranges

# Generate portion size ranges for selected categories
portion_ranges = generate_portion_ranges(selected_categories, base_portion, variation_factor)

# Display portion ranges as a DataFrame for clarity
portion_ranges_df = pd.DataFrame.from_dict(portion_ranges, orient="index", columns=["Min Portion (grams)", "Max Portion (grams)"])
portion_ranges_df.index.name = "Food Category"

# Save or inspect the portion ranges
print(portion_ranges_df)

                    Min Portion (grams)  Max Portion (grams)
Food Category                                               
apple_pie                           150                  263
baby_back_ribs                      116                  280
baklava                             122                  262
beef_carpaccio                      127                  273
beef_tartare                        135                  298
beet_salad                          139                  275
beignets                            111                  288
bibimbap                            101                  285
bread_pudding                       117                  288
breakfast_burrito                   113                  258
bruschetta                          126                  297
caesar_salad                        129                  272
cannoli                             108                  262
caprese_salad                       136                  290
carrot_cake             

# Generate placeholders for the calories information 

In [59]:
# Function to calculate calories based on portion size and calorie factor
def calculate_calories(portion_df, calorie_factors=None, default_calorie_factor=2.0):
    results = []
    for index, row in portion_df.iterrows():
        # Fetch min and max portion sizes
        min_portion = row["Min Portion (grams)"]
        max_portion = row["Max Portion (grams)"]
        
        # Fetch or generate calorie factor
        calorie_factor = calorie_factors.get(index, default_calorie_factor) if calorie_factors else default_calorie_factor
        
        # Calculate calories
        min_calories = min_portion * calorie_factor
        max_calories = max_portion * calorie_factor
        
        # Append to results
        results.append({
            "Food Category": index,
            "Min Portion (grams)": min_portion,
            "Max Portion (grams)": max_portion,
            "Min Calories": round(min_calories),
            "Max Calories": round(max_calories),
            "Calorie Factor (cal/g)": round(calorie_factor, 2)
        })
    
    return pd.DataFrame(results)

# Generate calorie factors dynamically (optional)
def generate_calorie_factors(categories, base=2.0, variation=1.0):
    return {category: np.random.uniform(base - variation, base + variation) for category in categories}

# Generate calorie factors
calorie_factors = generate_calorie_factors(selected_categories)

# Calculate calories using existing portion size data
calories_df = calculate_calories(portion_ranges_df, calorie_factors=calorie_factors)

# Display the final DataFrame
print(calories_df)


         Food Category  Min Portion (grams)  Max Portion (grams)  \
0            apple_pie                  150                  263   
1       baby_back_ribs                  116                  280   
2              baklava                  122                  262   
3       beef_carpaccio                  127                  273   
4         beef_tartare                  135                  298   
5           beet_salad                  139                  275   
6             beignets                  111                  288   
7             bibimbap                  101                  285   
8        bread_pudding                  117                  288   
9    breakfast_burrito                  113                  258   
10          bruschetta                  126                  297   
11        caesar_salad                  129                  272   
12             cannoli                  108                  262   
13       caprese_salad                  136     

# Out-of-Sample Testing with actual food images

In [66]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [68]:
# Path to the OOS data folder
oos_data_path = "/Users/angelaliang/Duke University /Duke Classes/F24/ME 555 CEE 590/Project/datasets/OOS data"

# Get all image file names in the folder
oos_images = [f for f in os.listdir(oos_data_path) if f.endswith(('.jpg', '.jpeg', '.png', '.webp', '.avif'))]
print(oos_images)

['ceviche.jpg', 'cheesecake.jpg', 'Baklava.jpg', 'Apple Pie.jpg', 'chicken_quesadilla.jpg', 'breakfast_burrito.jpg', 'Bibimbap.jpg', 'baby back ribs.jpg', 'beignets.jpg', 'cannoli.jpg', 'beef_carpaccio.jpg', 'caprese_salad.jpg', 'caesar_salad.jpg', 'beef_tartare.jpg', 'cheese_plate.jpg', 'chicken_curry.jpg', 'carrot_cake.jpg', 'beet_salad.jpg', 'bread_pudding.jpg', 'bruschetta.jpg']


In [69]:
oos_image_paths = [os.path.join(oos_data_path, img) for img in oos_images]
print(f"Found {len(oos_images)} OOS images for testing.")

Found 20 OOS images for testing.


In [73]:
# use the fine tuned model
# Load the fine-tuned model
model = load_model('fine_tuned_food20_model.h5')

# Preprocess function for OOS images
def preprocess_oos_image(img_path, target_size=(299, 299)):
    """
    Preprocesses an out-of-sample image for model inference and identifies any problematic files.
    Args:
        img_path: Path to the input image.
        target_size: Target size for the CNN model input.
    Returns:
        Preprocessed image array ready for prediction, or None if the file is problematic.
    """
    try:
        img = image.load_img(img_path, target_size=target_size)  # Resize the image
        img_array = image.img_to_array(img)  # Convert image to array
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array = preprocess_input(img_array)  # Normalize the input
        return img_array
    except Exception as e:
        print(f"Error processing file {img_path}: {e}")
        return None


# Check all files in OOS images and preprocess only valid ones
valid_images = []
valid_filenames = []

for img_path in oos_image_paths:
    preprocessed_img = preprocess_oos_image(img_path)
    if preprocessed_img is not None:  # Only append if preprocessing was successful
        valid_images.append(preprocessed_img)
        valid_filenames.append(os.path.basename(img_path))

print(f"Processed {len(valid_images)} valid images.")
print(f"Problematic files (if any): {len(oos_image_paths) - len(valid_images)}")


Processed 20 valid images.
Problematic files (if any): 0


### Perform Predictions on OOS Images

In [74]:
# Labels for your 20 categories
labels = selected_categories  # Already reduced to 20 categories

# Perform predictions
results = []
for img_path in oos_image_paths:
    img_name = os.path.basename(img_path)
    preprocessed_img = preprocess_oos_image(img_path)
    
    # Model prediction
    predictions = model.predict(preprocessed_img)
    predicted_label = labels[np.argmax(predictions)]
    
    # Append results
    results.append({"Image Name": img_name, "Predicted Category": predicted_label})

# Convert results to a DataFrame
oos_results_df = pd.DataFrame(results)
print(oos_results_df)


1/1 [==============================] - 0s 148ms/step
                Image Name  Predicted Category
0              ceviche.jpg             ceviche
1           cheesecake.jpg          cheesecake
2              Baklava.jpg             baklava
3            Apple Pie.jpg           apple_pie
4   chicken_quesadilla.jpg  chicken_quesadilla
5    breakfast_burrito.jpg   breakfast_burrito
6             Bibimbap.jpg            bibimbap
7       baby back ribs.jpg      baby_back_ribs
8             beignets.jpg            beignets
9              cannoli.jpg             cannoli
10      beef_carpaccio.jpg       chicken_curry
11       caprese_salad.jpg       caprese_salad
12        caesar_salad.jpg        caesar_salad
13        beef_tartare.jpg        beef_tartare
14        cheese_plate.jpg        cheese_plate
15       chicken_curry.jpg       chicken_curry
16         carrot_cake.jpg         carrot_cake
17          beet_salad.jpg      beef_carpaccio
18       bread_pudding.jpg       bread_pudding
19     

In [75]:
# Merge predicted categories with portion size and calorie data
oos_results_with_calories = oos_results_df.merge(calories_df, 
                                                 left_on='Predicted Category', 
                                                 right_on='Food Category', 
                                                 how='left')

# Drop duplicate columns and display
oos_results_with_calories = oos_results_with_calories.drop(columns=["Food Category"])
print(oos_results_with_calories)


                Image Name  Predicted Category  Min Portion (grams)  \
0              ceviche.jpg             ceviche                  144   
1           cheesecake.jpg          cheesecake                  114   
2              Baklava.jpg             baklava                  122   
3            Apple Pie.jpg           apple_pie                  150   
4   chicken_quesadilla.jpg  chicken_quesadilla                  125   
5    breakfast_burrito.jpg   breakfast_burrito                  113   
6             Bibimbap.jpg            bibimbap                  101   
7       baby back ribs.jpg      baby_back_ribs                  116   
8             beignets.jpg            beignets                  111   
9              cannoli.jpg             cannoli                  108   
10      beef_carpaccio.jpg       chicken_curry                  116   
11       caprese_salad.jpg       caprese_salad                  136   
12        caesar_salad.jpg        caesar_salad                  129   
13    

In [76]:
# Extract Ground Truth Labels from File Names
def extract_ground_truth(image_name):
    
    """
    Extracts the ground truth label from the image file name.
    Args:
        image_name: Name of the image file (e.g., 'Apple Pie.jpg').
    Returns:
        Ground truth label (e.g., 'apple_pie').
    """
    return image_name.split('.')[0].replace(' ', '_').lower()


In [77]:
# Add ground truth labels to the DataFrame
oos_results_with_calories['Ground Truth'] = oos_results_with_calories['Image Name'].apply(extract_ground_truth)

# Check the updated DataFrame
print(oos_results_with_calories[['Image Name', 'Predicted Category', 'Ground Truth']])


                Image Name  Predicted Category        Ground Truth
0              ceviche.jpg             ceviche             ceviche
1           cheesecake.jpg          cheesecake          cheesecake
2              Baklava.jpg             baklava             baklava
3            Apple Pie.jpg           apple_pie           apple_pie
4   chicken_quesadilla.jpg  chicken_quesadilla  chicken_quesadilla
5    breakfast_burrito.jpg   breakfast_burrito   breakfast_burrito
6             Bibimbap.jpg            bibimbap            bibimbap
7       baby back ribs.jpg      baby_back_ribs      baby_back_ribs
8             beignets.jpg            beignets            beignets
9              cannoli.jpg             cannoli             cannoli
10      beef_carpaccio.jpg       chicken_curry      beef_carpaccio
11       caprese_salad.jpg       caprese_salad       caprese_salad
12        caesar_salad.jpg        caesar_salad        caesar_salad
13        beef_tartare.jpg        beef_tartare        beef_tar

In [78]:
# Calculate the number of correct predictions
oos_results_with_calories['Correct'] = oos_results_with_calories['Predicted Category'] == oos_results_with_calories['Ground Truth']

# Calculate accuracy
accuracy = oos_results_with_calories['Correct'].mean()
print(f"Model Accuracy on OOS Images: {accuracy * 100:.2f}%")

Model Accuracy on OOS Images: 90.00%
